In [1]:
# Import necessary modules
import os
import pandas as pd
from dotenv import load_dotenv
from modules.paragraph_generation import generate_paragraph, generate_advertising_paragraph
from modules.data_processing import load_and_filter_data, preprocess_columns_simple, enrich_advertiser_data_with_domain
from modules.linkedin_requests import initialize_client, generate_linkedin_request, generate_followup_message

# Load API key
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
pd.set_option('display.expand_frame_repr', False)


In [2]:
# Load and preprocess datasets
people_path = './data/YC F24 Founder People.csv'
personal_data_path = './data/Sudarshan Sridharan People.csv'

relevant_columns = [
    'full_name', 'title', 'location', 'company', 'company_linkedin',
    'company_description', 'sub_title', 'summary', 'current_job_description',
    'person_industry', 'skills', 'education', 'work_history', 'company_domain'
]

people_data = load_and_filter_data(people_path, relevant_columns)
people_data = preprocess_columns_simple(people_data)

personal_data = load_and_filter_data(personal_data_path, relevant_columns)
personal_data = preprocess_columns_simple(personal_data)

# Enrich personal data with domain information
personal_data = enrich_advertiser_data_with_domain(personal_data)

# Generate paragraphs
people_data['descriptive_paragraph'] = people_data.apply(generate_paragraph, axis=1)
personal_data['advertising_paragraph'] = personal_data.apply(generate_advertising_paragraph, axis=1)


In [3]:
personal_data

,full_name,title,location,company,company_linkedin,company_description,sub_title,summary,current_job_description,person_industry,skills,education,work_history,company_domain,domain_info,advertising_paragraph
0,Sudarshan Sridharan,Founder,"San Francisco Bay, San Francisco Bay Area, Uni...",Pipeline,https://www.linkedin.com/company/withpipeline,"Get more leads, close more deals. Pipeline set...","CEO, Pipeline",NaN,Pipeline puts LinkedIn prospecting and outreac...,"Technology, Information and Internet",Investing,Clemson University,"Pipeline, Fion Technologies, SaveMAPS, Second ...",withpipeline.com,Sign Up → Sign Up → NEW Supercharge Your Sales...,Sudarshan Sridharan is the Founder at Pipeline...


In [4]:
# Load the system prompt from a .txt file
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()

# Initialize Anthropic client
client = initialize_client(ANTHROPIC_API_KEY)

# Use the first advertiser's data for testing
advertising_paragraph = personal_data['advertising_paragraph'].iloc[0]
domain_info = personal_data['domain_info'].iloc[0]

# Process only the first 10 entries
max_entries = 25

results = []

print("Generated LinkedIn Requests and Follow-Up Messages:\n" + "=" * 50)

for idx, row in people_data.iterrows():
    if idx >= max_entries:
        break
    
    descriptive_paragraph = row['descriptive_paragraph']
    
    # Generate the connection request
    connect_note = generate_linkedin_request(client, advertising_paragraph, descriptive_paragraph, domain_info, system_prompt)
    
    # Generate the follow-up message
    follow_up_message = generate_followup_message(client, advertising_paragraph, descriptive_paragraph, domain_info, system_prompt)
    
    # Collect results
    results.append({
        'Name': row['full_name'],
        'Connect Note': connect_note,
        'Follow-Up Message': follow_up_message
    })

# Create DataFrame from results
output_df = pd.DataFrame(results)

# Display the DataFrame
output_df.style.set_properties(**{'text-align': 'left'})

# Save the DataFrame to a CSV for reference
output_df.to_csv('linkedin_messages.csv', index=False)

Generated LinkedIn Requests and Follow-Up Messages:


In [5]:
output_df.style

,Name,Connect Note,Follow-Up Message
0,Edward Aryee,"Hi Edward! As a fellow YC founder, I love seeing other builders tackling B2B automation. Your background at Google and Microsoft, plus your work on SRE.ai caught my eye. Would love to connect and hear more about your journey building in the AI space - especially given your experience with scaled systems.","Hey Edward! Thanks for connecting. Love seeing another Bay Area founder who's also worked at some incredible tech companies (Google and Microsoft - quite the resume!). I noticed we're both focused on making tech teams more efficient - you with SRE.ai and me with sales automation. Having worked with several early-stage startups, I've seen how much time technical teams spend on repetitive tasks instead of core innovation. We've built Pipeline to help teams automate their LinkedIn outreach and keep their CRM data fresh without the manual work. Since you're building in the SRE space, I thought you might find it interesting - especially as you scale your YC company. Would love to learn more about your automation challenges at SRE.ai and share how other YC founders have streamlined their outreach. Want to hop on a quick call next week? Happy to share some specific examples of how other technical founders are using automation to grow while staying focused on product development. Best, Sudarshan --- Note: This response keeps a friendly tone while being specific about shared context (Bay Area, tech background, YC connection). It introduces Pipeline naturally by connecting it to the recipient's interests in automation and efficiency, without being overly pushy. The message acknowledges his impressive background while offering valuable insights relevant to his current role."
1,Victor Cheng,"Hey Victor! As another founder working with early-stage startups, I'm really curious about what you're building at vly.ai - especially given your experience at Quantum Labs and Hatch Coding. Would love to connect and hear about your YC journey so far!","Hi Victor! Thanks for connecting. Saw you're working on vly.ai and got into YC F24 (congrats on that btw!). As someone who's also building tools to help businesses grow, I've been thinking about how AI is reshaping the way companies handle operations. Given your experience with both early-stage startups and established companies like Autodesk, I'd be curious to hear your thoughts on automation in the sales and customer engagement space. We've been helping B2B startups streamline their outbound processes at Pipeline, particularly around LinkedIn engagement and CRM automation. Since you're in the AI space and likely thinking about scaling your own outreach soon, I'd love to share how we're approaching this challenge - might be some interesting parallels with what you're building at vly.ai. Would you be open to a quick chat next week? Happy to share what we've learned from working with other YC companies, and I'm genuinely interested in learning more about your vision for vly.ai. Best, Sudarshan Note: This message maintains a professional but friendly tone, acknowledges their YC connection, and relates Pipeline's work to their context without being overly pushy. It shows genuine interest in their work while naturally introducing the potential value proposition."
2,Daryl Budiman,"Hey Daryl! Really cool to see what you're building with Andoria coming out of YC F24. As a fellow startup founder in SF working on sales tools, I'd love to connect and chat about scaling early-stage companies. Your experience at Amazon and Evernote must give you some unique insights!","Hey Daryl! Thanks for connecting. I noticed you're building something exciting with Andoria in the YC F24 batch (congrats on getting in!). As a fellow startup founder who's worked with several early-stage companies, I've seen firsthand how crucial efficient outbound sales can be for B2B growth, especially during the critical early stages. At Pipeline, we've developed tools that automate LinkedIn